# Hardware considerations and limitations for classical feedforward and control flow

<details>
<summary><b>Package versions</b></summary>

The code on this page was developed using the following requirements.
We recommend using these versions or newer.

```

```
</details>

[Classical feedforward and control flow](/guides/classical-feedforward-and-control-flow) shows how to use Qiskit to build circuits that involve classical feedforward and control flow, also known as dynamic circuits. When actually running such circuits on quantum hardware, there are several considerations and limitations to be aware of. Many of these limitations exist because the underlying technology supporting these features is in an early stage of development, and we hope to be able to address them in the future.


## Memory limits and latency in control hardware

![Diagram showing control hardware architecture](/images/guides/rta-architecture.avif)

Running circuits on quantum processors involves not only the qubits themselves, but also a system of classical electronics and computers to generate and receive waveforms and orchestrate the control logic. When a job is submitted to the IBM Quantum&reg; service, it is processed into multiple classical programs that must be distributed between two kinds of units: central controllers and qubit controllers (see diagram above). A job may fail if it exceeds certain limitations of these controllers. There are two kinds of limitations to be aware of:

- **Limited working memory**. This primarily affects the central controllers, and jobs will fail if they cause this memory limit to be exceeded.
- **Latency caused by classical computation**. Running circuits that use classical feedforward and control flow involves performing classical computation during the course of the circuit execution. Due to the limited coherence time of qubits, there is a limited time budget for performing these computations. A job may fail at compile time if the compilation detects that the classical computation overhead is too large.

The memory requirements and classical latencies of a job are affected by the following factors:

- **Number of circuits**. When multiple circuits are submitted in a single job, they become concatenated into a single large circuit, with qubit initialization operations between them. Qubit initialization is implemented as a conditional reset on all qubits used in the large circuit.
  - Central controller: Memory usage scales proportionally with the number of circuits.
- **Amount of control flow**.
  - Central controller: Memory usage scales proportionally with the number of control flow decisions.
  - Qubit controller: A control flow construct with too many or too large logic branches may not be realizable.
- **Resets**.
  - Central controller: Memory usage scales proportionally with the number of resets.
- **Measurements**.
  - Central controller: Memory usage scales proportionally with the number of measurements used by the central controller for control flow.
<span id="buffer-overflow"></span>
- **Buffer overflow error**. One of the most common limitations users can encounter when executing dynamic circuits is a buffer overflow during measurement result collection. The measurement buffer on the control system can only store a limited number of measurement results. If the circuit performs many measurements in rapid succession, the buffer might fill before the control system has time to stream the results to the host, causing a job failure.

  If you encounter the error `"RuntimeJobFailureError: unable to retrieve job result. A buffer overflow occurred during result collection..."`, try these strategies to mitigate this issue:
  - **Increase `rep_delay` (delay between shots):** This adds time between shots, giving the control system more time to offload measurement results to the host and clear the buffer. This is the most direct fix and does not impact shot quality, though it will increase total job execution time.
  - **Insert `qc.delay()` after measurements:** Add a delay instruction between measurements within a circuit to give the system time to stream results and prevent the buffer from filling.
  **Note:** this approach increases the circuit duration, which might reduce shot quality due to qubit decoherence.

  For more information, see the [Fixed and dynamic repetition rate execution](/guides/repetition-rate-execution) guide, which explains dynamic repetition rate execution and how to set `rep_delay`.